In [5]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Using cached google_auth-2.40.3-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached requests-2.32.4-py3-none-any.whl.metadata (4.9 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached charset_normalizer-3.4.2-cp313-cp313-macosx_10_13_universal2.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.7.9-py3-none-any.whl.metadata (2.4 kB)
  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
labels = ["Entertainment", "Study", "Motivation", "How To", "Money & Carrer", "Gaming"]

In [ ]:
# from googleapiclient.discovery import build

# API_KEY = ''
# youtube = build('youtube', 'v3', developerKey=API_KEY)

# def get_videos(query, max_results=50):
#     request = youtube.search().list(
#         q=query,
#         part='snippet',
#         type='video',
#         maxResults=max_results
#     )
#     response = request.execute()
#     videos = []
#     for item in response['items']:
#         title = item['snippet']['title']
#         description = item['snippet']['description']
#         videos.append({'title': title, 'description': description})
#     return videos

# # Example: Get 50 "tutorial" videos
# tutorial_videos = get_videos('tutorial', 50)

In [ ]:
from googleapiclient.discovery import build
import time

API_KEY = '*****************************'
youtube = build('youtube', 'v3', developerKey=API_KEY)

# Define your categories and relevant search queries for each
categories = {
    "Entertainment": ["entertainment", "comedy", "funny", "music video", "movie trailer"],
    "Study": ["study", "study with me", "study tips", "study motivation", "study techniques", 'learn'],
    "Motivation": ["motivation", "motivational speech", "inspiration", "self improvement", 'productivity', 'mindset'],
    "How To": ["how to", "tutorial", "guide", "step by step", "DIY"],
    "Money & Career": ["finance", "career advice", "investing", "money tips", "job interview", "startup"],
    "Gaming": ["gaming", "let's play", "gameplay", "video game", "esports"]
}

videos_per_category = 150
max_results_per_query = 50  # YouTube API max per request

all_videos = []

def get_videos(query, label, max_results):
    collected = 0
    next_page_token = None
    videos = []
    while collected < max_results:
        request = youtube.search().list(
            q=query,
            part='snippet',
            type='video',
            maxResults=min(max_results_per_query, max_results - collected),
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            title = item['snippet']['title']
            description = item['snippet']['description']
            videos.append({'title': title, 'description': description, 'label': label})
            collected += 1
            if collected >= max_results:
                break
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
        time.sleep(0.5)  # To respect API quota
    return videos

for label, queries in categories.items():
    category_videos = []
    for query in queries:
        # Divide quota per query to reach ~150 per category
        quota = videos_per_category // len(queries)
        videos = get_videos(query, label, quota)
        category_videos.extend(videos)
        if len(category_videos) >= videos_per_category:
            break
    # If still short, fill with more from the first query
    if len(category_videos) < videos_per_category:
        extra_needed = videos_per_category - len(category_videos)
        extra_videos = get_videos(queries[0], label, extra_needed)
        category_videos.extend(extra_videos)
    all_videos.extend(category_videos[:videos_per_category])
    print(f"Collected {len(category_videos[:videos_per_category])} videos for {label}")

# Now all_videos is a list of dicts with title, description, and label
print(f"Total videos collected: {len(all_videos)}")


Collected 150 videos for Entertainment
Collected 150 videos for Study
Collected 150 videos for Motivation
Collected 150 videos for How To
Collected 150 videos for Money & Career
Collected 150 videos for Gaming
Total videos collected: 900


In [8]:
import json

with open('data.json', 'r') as f:
    all_videos = json.load(f)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

texts = [video['title'] + ' ' + video['description'] for video in all_videos]
labels = [video['label'] for video in all_videos]

model = make_pipeline(TfidfVectorizer(), LinearSVC())
model.fit(texts, labels)

,steps,"[('tfidfvectorizer', ...), ('linearsvc', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [10]:
prediction = model.predict(["BUILDING A £400,000 MCLAREN FROM TEMU"])[0]

In [11]:
prediction

np.str_('Money & Career')